In [4]:
import json
import pandas as pd
import re
from datetime import datetime

#Converts json to a list
with open('transaction-data-adhoc-analysis.json') as f:
    data = json.load(f)
transactionitems=[]
product = []
udata = pd.read_json('transaction-data-adhoc-analysis.json', orient='column')
df = pd.DataFrame(udata)

for index, row in df.iterrows(): # seperates the groups of purchases into rows 
    transaction = row['transaction_items']
    date = row['transaction_date']
    item = transaction.split(";") #makes a dictionary for each group of items
    for i in range(0,len(item)): #splits each item bought from its group and adds a date component
        itemtobeadded = item[i] +',' + date
        transactionitems.append(itemtobeadded)   

#splits purchases into columns by Brand, Name, and Quantity
for v in range(0, len(transactionitems)):
    fitem = transactionitems[v].split(',')
    product.append(fitem)

#creates new dataframe for brand name, product name, quantity, and date
ff = pd.DataFrame(product,columns = ["Brand Name", "Product Name", "Quantity", "Date"])

#iterates through every item in column Quantity and removes all elements that is not within 0-9
for y in range(len(ff['Quantity'])):
    ff.loc[y, 'Quantity'] = re.sub('[^0-9]','',(str(ff.loc[y, 'Quantity'])))

#converts to int column 'quantity'
ff["Quantity"] = ff["Quantity"].astype(int)

#gets the month component of the date of each transaction
def gmonth(Date):
    datem = datetime.strptime(Date, "%Y/%m/%d")
    return datem.month
ff["Date"] = ff["Date"].apply(gmonth)

In [7]:
fdf = pd.DataFrame()

#prices of each item
prices = [1299,1500,150,799,1990,199,500]

#return the quantity of each item per month
for i in range(1,7):
    month = ff.loc[ff["Date"] == i]
    month.groupby(['Product Name']).sum().squeeze().drop("Date", axis= 1)
    fdf[i] = month.groupby(['Product Name']).sum().squeeze().drop("Date", axis= 1)

#Quantity of each item per month
ChicharonQ = fdf.iloc[0]
VitaminsQ = fdf.iloc[1]
WormsQ = fdf.iloc[2]
KimchiQ = fdf.iloc[3]
MilkQ = fdf.iloc[4]
OrangeQ = fdf.iloc[5]
VegetablesQ = fdf.iloc[6]

#Multiplies each quantity to the price of each item
Chicharon = fdf.iloc[0] * prices[0]
Vitamins = fdf.iloc[1] * prices[1]
Worms = fdf.iloc[2] * prices[2]
Kimchi = fdf.iloc[3] * prices[3]
Milk = fdf.iloc[4] * prices[4]
Orange = fdf.iloc[5] * prices[5]
Vegetables = fdf.iloc[6] * prices[6]


Orange

1    1945026
2    1997363
3    1912589
4    1972886
5    1982836
6    2011094
Name: Orange Beans, dtype: int32